# PROJECT O4: DOCUMENTS CLASSIFICATION WITH LINEAR REGRESSION
----
##### Giảng viên hướng dẫn: ***Trần Thị Thảo Nhi***.
##### Sinh viên thực hiện: ***Võ Trần Quang Tuấn*** - ID: ***18127248***
##### Môn học: Toán ứng dụng và thống kê - MTH00051(18CLC2).
##### Trường đại học Khoa học Tự Nhiên - ĐHQG TPHCM.
##### Khoa Công nghệ Thông tin - FIT HCMUS.
----

## PROPLEM DESCRIPTIONS
> Xử lý dữ liệu dạng văn bản.  
Sử dụng mô hình hồi quy tuyến tính dùng bình phương tối tiểu.  
Sử dụng độ chính xác để đánh giá mô hình.


## CODE - REPORT DESCRIPTIONS
> Report đi kèm với file .ipynb, được viết dạng markdown.  
Code python được viết trên từng cell, theo từng quá trình xử lý và chức năng nhất định.  
Để run, chạy cell by cell và kiểm tra kết quả.


## HIỆN THỰC HOÁ QUÁ TRÌNH

###I. Import các thư viện cần thiết cho việc phân tích văn bản.

In [ ]:
# DON'T CHANGE this part: import libraries
import numpy as np
import scipy
import json
from nltk.stem import PorterStemmer 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import re
import itertools

### II. Phần nhập tên file train và file validation test:

In [ ]:
# DON'T CHANGE this part: read data path
train_set_path, valid_set_path, random_number = input().split()

train.json valid.json 10


### III. Preprocessing.
> Converting text to lower case.  
Converting number to `num`.  
Tokenization.    
Removing stopword.  
Stemming.

#### 1. Đọc file dữ liệu dạng json, tách thành hai trường: overall và review.
- overall: danh sách các đánh giá của người dùng (từ 1 -> 5), dạng `float`.
- review: danh sách các review của người dùng, có dạng là `string`

In [ ]:
def read_data(file_path):
  data_file = open(file_path, 'r')
  data = json.loads(data_file.read())
  overall = [item['overall'] for item in data]
  review = [item['reviewText'] for item in data]
  return overall, review

#### 2. Chuyển hoá nhãn thành vector.
- Vì mỗi label đều ở dạng một con số `float`, cho nên ta phải chuyển về một vector 5 chiều tương ứng với 5 loại đánh giá từ 1->5.
- Mỗi vector có 5 phần tử, đánh giá thuộc loại nào thì phần tử có chỉ số point-1 sẽ có giá trị là 1, các phần tử còn lại bằng 0, với point là overall.
- Trả về ma trận label có chiều `Nx5` với N là số người tham gia đánh giá.  
- ***ĐÂY CHÍNH LÀ KĨ THUẬT ONE-HOT ENCODING***

In [ ]:
def convert_label(lst_overall):
  N = len(lst_overall)
  label_matrix = np.zeros((N, 5))
  for index in range(N):
    label = int(lst_overall[index])
    label_matrix[index, label-1] = 1
  return label_matrix

#### 3. Xử lý text dạng `string`.
- Chuyển toàn bộ text thành chữ thường.
- Tách token.
- Xoá stopwords trong tiếng anh.
- Chuyển các từ dẫn xuất thành nguyên mẫu từ (stemming).
- Chuyển số thành `num`.

In [ ]:
def preprocessing(text):
  # lower sentence
  text = text.lower()
  # tokenize
  words = word_tokenize(text)
  # remove stopwords
  en_stops = set(stopwords.words('english'))
  words_new = []
  for i in range(len(words)):
    if words[i] not in en_stops:
      words_new.append(words[i])
  # stemming
  st = PorterStemmer()
  for i in range(len(words_new)):
    if words_new[i].isnumeric():
      words_new[i] = 'num'
    else:
      words_new[i] = st.stem(words_new[i])
  return words_new

#### 4. Xử lý text trong tập valid.json.
- Những từ nào chưa có trong tập vocab thì chuyển thành `unk`
- Tiền xử lý như các đoạn text trong tập train.
- vocab thuộc kiểu `set`, chứa các từ của tập train trong corpus.

In [ ]:
def valid_text_processing(text, vocab):
  tokens = preprocessing(text)
  for i in range(len(tokens)):
    if tokens[i] not in vocab:
      tokens[i] = 'unk'
  return tokens

#### 5. Tạo tập vocab.
- Sử dụng text trong tập train để tạo tập vocab.
- Vocab sẽ thuộc kiểu `set`, mục đích để truy xuất đến các phần tử nhanh hơn.

In [ ]:
def make_vocab(training_set):
  vocab = set()
  for text in training_set:
    tokens = preprocessing(text)
    for token in tokens:
      vocab.add(token)
  return vocab

#### 6. Tạo histogram vector cho từng đoạn text review, và tạo histogram matrix.
- `make_histogram_vector(text, vocab)`: tạo vector histogram cho từng đoạn text
- `embedding(dataset, vocab)`: xây dựng ma trận histogram với tập vocab và dataset.

In [ ]:
def make_histogram_vector(text, vocab):
  tokens = valid_text_processing(text, vocab)
  vocab_size = len(vocab)
  fre = np.zeros(vocab_size)
  i = 0
  for word in vocab:
    fre[i] = tokens.count(word)
    i += 1
  return fre/np.sum(fre)

def embedding(dataset, vocab):
  histogram_matrix = []
  for text in dataset:
    fre = make_histogram_vector(text, vocab)
    histogram_matrix.append(fre)
  return np.array(histogram_matrix)

### IV. Quá trình huấn luyện model.
> Cài đặt hàm hồi quy tuyến tính.  
Tìm tham số hồi quy.

#### 1. Cài đặt hàm tính tham số của model hồi quy tuyến tính.
- Ma trận input đầu vào sẽ có dạng `Nxn`, với N là số đoạn text trong dataset, n là số phần tử tập vocab.
- Tham số theta cần huấn luyện có dạng `nx5`, n là số phần tử tập vocab, model này sẽ trả về một vector 5 chiều tương ứng với 5 loại đánh giá.
- Ma trận nhãn (label) có dạng `Nx5` với N là số đoạn text trong dataset.


In [ ]:
def linear_regression(dataset, label):
  return np.linalg.pinv(dataset) @ label

#### 2. Tìm tham số cho model M2.
- Sử dụng model hồi quy tuyến tính.
- Dataset chứa các review text trong tập training đã được chuyển thành dạng histogram vector.
- Label được chuyển thành dạng one-hot encoding.

> Chuẩn bị data và label cho việc training.

In [ ]:
overall, review = read_data(train_set_path)
label_matrix = convert_label(overall)
vocab = make_vocab(review)
histogram_matrix = embedding(review, vocab)

> Huấn luyện model, tìm tham số model.

In [ ]:
param = linear_regression(histogram_matrix, label_matrix)

### V. Kiểm thử độ chính xác của model.
> Dùng hàm softmax để quyết định nhãn của text.  
Hàm softmax `scipy.special.softmax` sẽ trả ra một phân phối xác suất của text vào 5 lớp. Chọn lớp có xác suất cao nhất.  

#### 1. Cài đặt hàm softmax để trả về label của text trong valib.json
- Hàm `softmax` sẽ trả về phân phối xác suất của text vào 5 class.   
- Dùng `np.argmax` để trả về label của class có xác suất cao nhất. 

In [ ]:
def softmax(x):
  return np.exp(x)/np.sum(np.exp(x))

def define_label(x):
  distribution = softmax(x)
  return np.argmax(distribution) + 1

#### 2. Cài đặt hàm tính accuracy của model: lấy tổng số text phân lớp đúng/số text đem kiểm thử.

In [ ]:
def accuracy(y_predict, overall):
  count = 0
  for i in range(len(y_predict)):
    label_predict = define_label(y_predict[i])
    if int(label_predict) == int(overall[i]):
      count += 1
  return count/len(overall)

#### 3. Kiểm thử model trên tập valid.json
> Chuẩn bị dữ liệu.


In [ ]:
overall_valid, review_valid = read_data(valid_set_path)
histogram_matrix_valid = embedding(review_valid, vocab)

> Tính label dự đoán.

In [ ]:
y_predict = np.dot(histogram_matrix_valid, param)

> Tính accuracy.

In [ ]:
acc = accuracy(y_predict, overall_valid)

## OUTPUT THU ĐƯỢC

In [ ]:
print(valid_text_processing(review_valid[int(random_number)], vocab))
print(f'M2 - {acc}')

['brought', 'back', 'num', 'player', 'mode', 'like', 'mario', 'parti', 'num', '&', 'num']
M2 - 0.522
